In [30]:
#functions

from datetime import datetime as dt
from datetime import date

#change directory path
def change_path(minslice,newpath):
    import os
    os.getcwd()
    path = os.getcwd()
    writepath = path[:minslice] + newpath
    return writepath

# Takes a date as a datetime, and returns age. 
# If there is no date given, returns None
def calculate_age(born):
    today = date.today()
    if born == None:
        return None
    else:
        return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '' or date == '0000-00-00' or date == 'NULL':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer and or float,
# and returns an int or None.
def parse_maybe_int(i, typ):
    if i == '' or i == None or i == 'NULL':
        return None
    else:
        if typ == 'int':
            return int(float(i))
        elif typ == 'float':
            return float(i)

#csv reader to dictionary
def CsvReader(file):
    import csv
    with open(file) as f:
        reader = csv.reader(f)
        header = next(reader)
        a = [dict(zip(header, map(str, row))) for row in reader]
    return a

#Write dictionary to csv
def CSVWriter(csv_file,csv_columns,dict_data):
    import csv
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)

#function to get column names
def get_col_name(data):
    return list(data[0].keys())

#delete one column for csv dictionary
def DeleteColumn(table,col):
    for row in table:
        del row[col]
    return table

#delete various columns for csv dictionary
def DeleteColumns(table, cols):
    for i in range(len(cols)):
        table = DeleteColumn(table,cols[i])

#Create new field in list of dictionaries from another list of dictionaries        
def new_field_from_another(list1, list2, id1, id2, new):
    for row in list1:
        for record in list2:
            if record[id2] == row[id1]:
                record[new] = row[new]
    return list2

#get index of item in a list
def get_index(alist, item):    
    for i, j in enumerate(alist):
        if j == item:
            return i
        
#Append column to list from list of dictionaries
def AppendCol(alist,item):
    newlist = []
    for row in alist:
        newlist.append(row[item])
    return newlist

In [31]:
#read csv
jobseekers = CsvReader('20180515_jobseekers.csv')
CvEmpHist = CsvReader('20180515_CvEmpHist.csv')
University = CsvReader('20180515_University.csv')

In [32]:
#Delete unnessesary columns from jobseeker data
jobseekers_CTD = ['Address','City','Nationality','About','JobAlert','ProfilePhoto','Provider','jobtitle','Category','signuptype','alertStatus','confirm_code','confirmed']

DeleteColumns(jobseekers, jobseekers_CTD)

#Delete unnessesary columns from employment history data
CvEmpHist_CTD = ['Id','Employer','Jobtitle','Startdate','Enddate','Location','description']

DeleteColumns(CvEmpHist, CvEmpHist_CTD)

#Delete unnessesary columns from University data
University_CTD = ['Id','University','StartDate','EndDate','GPA','OutOf','Course','projects']

DeleteColumns(University, University_CTD)


In [33]:
# Clean up the data types in the jobseekers table
for jobseeker in jobseekers:
    jobseeker['DOB'] = parse_maybe_int(calculate_age(parse_date(jobseeker['DOB'])),'int')
    jobseeker['yr_exp'] = parse_maybe_int(jobseeker['yr_exp'],'int')
    jobseeker['no'] = parse_maybe_int(jobseeker['no'],'int')
    jobseeker['RegDate'] = parse_date(jobseeker['RegDate'])

#jobseekers[0]    

In [34]:
# Clean up the data types in the employment history table
for history in CvEmpHist:
    history['Salary'] = parse_maybe_int(history['Salary'],'float')
    history['Experience'] = parse_maybe_int(history['Experience'],'int')
    history['no'] = parse_maybe_int(history['no'],'int')

    
#CvEmpHist[0]

In [35]:
# Clean up the data types in the employment history table
for EduLevel in University:
    EduLevel['no'] = parse_maybe_int(EduLevel['no'],'int')

    
#University[0]

In [36]:
#add up all experience of each jobseeker's employment history and add to that particular jobseekers
for row in jobseekers:
    exp = 0
    for record in CvEmpHist:
        if record['no'] == row['no'] and record['Experience'] != None:
            exp += record['Experience']
    row['Experience'] = exp   

#jobseekers

In [37]:
#add up all experience of each jobseeker's employment history and add to that particular jobseekers
for row in jobseekers:
    spec = []
    for record in CvEmpHist:  
        if record['no'] == row['no'] and record['Specialization'] != '':
            spec.append(record['Specialization'])
    
    spec_set = list(set(spec))
    item_freq = 0 
    mf = ''
    for i in range(len(spec_set)):
        if item_freq < spec.count(spec_set[i]):
            item_freq = spec.count(spec_set[i])
            mf = spec_set[i]
    row['Specialization'] = mf


#jobseekers


In [38]:
#read degree data from CSV
degrees = CsvReader(change_path(52,'1. Original Data/degree.csv'))
#degrees

In [39]:
#Append Degree to list
degreeList = AppendCol(degrees,'degree')


In [40]:
#add up all experience of each jobseeker's employment history and add to that particular jobseekers
for row in jobseekers:
    levels = []
    for record in University:  
        if record['no'] == row['no'] and record['Courselevel'] != '':
            levels.append(record['Courselevel'])
    
    levels_set = list(set(levels))
    item_index = 0
    mf = ''
    for i in range(len(levels_set)):
        if item_index <= get_index(degreeList, levels_set[i]):
            item_index = get_index(degreeList, levels_set[i])
            mf = levels_set[i]
    row['Courselevel'] = mf


#jobseekers


In [41]:
#write list of dictionaries to csv file
#CSVWriter(change_path(52,'2. Prepared Data/20180517_jobseekers.csv'),get_col_name(jobseekers),jobseekers)

In [42]:
#CSVWriter(change_path(52,'4. Analysis/20180517_jobseekers.csv'),get_col_name(jobseekers),jobseekers)